# Units and Conversions

```{note}
MagentroPy uses the {mod}`pint` package internally for unit conversions, as well as the
{{ pint_pandas }} package to extend functionality to {{ DataFrame }}s. If you happen to be
using {{ pint_pandas }} as well, be aware that {{ MagentroData }}'s {class}`pint.UnitRegistry`
is set to `pint_pandas.PintType.ureg`, and it is reset with the necessary definitions and
conversion contexts each time conversions are performed.
```

````{tip}
Raw data units can be set during instantiation:
```python
magdata = MagentroData(..., raw_data_units={...})
```
````

In [ ]:
from IPython.display import display, HTML
from magentropy import MagentroData

magdata = MagentroData('magdata.csv', qd_dat=False, comment_col=None, T='T', H='H', M='M', M_err='M_err')
magdata.process_data()

Above, we've read a `.csv` file and processed the data. The default sample mass of 1.0 mg was used.
Luckily, the sample mass and all of the raw data units can be set at any time, before or after
processing, with the appropriate conversions applied automatically.

Right now, the data looks like this:

In [ ]:
def data_heads():
    display(HTML('<p>Raw data:</p>'))
    display(magdata.raw_df_with_units.head(3))
    display(HTML('<p>Converted data:</p>'))
    display(magdata.converted_df_with_units.head(3))
    display(HTML('<p>Processed data:</p>'))
    display(magdata.processed_df_with_units.head(3))

data_heads()

## Sample mass

To change the sample mass, use the {{ sample_mass }} attribute:

In [ ]:
magdata.sample_mass = 0.1
data_heads()

All of the per-mass columns (`'M_per_mass'`, `'M_per_mass_err'`, `'dM_dT'`, and `'Delta_SM'`)
have increased by a factor of 10.

Similarly, one can change the units:

In [ ]:
magdata.set_raw_data_units(sample_mass='g')
data_heads()

The per-mass columns have now decreased by a factor of 1000.

{mod}`pint`'s unit string parsing is used to change the units.
The logged output confirms that the unit change is as expected.

In [ ]:
magdata.set_raw_data_units(sample_mass='gram')
magdata.set_raw_data_units(sample_mass='grams')

Both the magnitude and the units can be set at once using {{ sample_mass_with_units }}:

In [ ]:
magdata.sample_mass_with_units

In [ ]:
magdata.sample_mass_with_units = (0.5, 'milligrams')
data_heads()

In [ ]:
magdata.sample_mass_with_units

## Column units

The input temperature, magnetic field, and magnetic moment units can be changed
using {{ set_raw_data_units }}. The moment error column will get the same units
as the moment column. The last four columns are fixed in cgs units.

In [ ]:
magdata.set_raw_data_units(T='degC', H='G', M='erg/G')
data_heads()

A few things to note:

1. The short string for "degrees Celcius" is `'degC'`; `'C'` stands for "Coulombs".

2. Changing the input units does not convert the input values themselves. Rather, it
determines how they are converted to cgs and SI units. For example, the converted and
processed data now begin at about 274.15 K. (Compare to the previous data output.)

3. In a vacuum, the conversion from Gauss to Tesla is the same as that from Oersted to Tesla
(divide by 10000), so the magnetic field column is unchanged.

4. Similarly, 1 emu is defined to be 1 erg/Gauss in the context of measuring magnetic moments,
so the magnetic moment column is unchanged.

5. Concluding the perhaps underwhelming lack of changes in the data, the unit scale is the same
for Celcius as it is for Kelvin, so the `'dM_dT'` and `'Delta_SM'` columns, which depend only on
the derivative with respect to temperature, are unchanged.

## Plotting

Temperature and field units will be reflected in the default plot labels and
titles when plotting the raw data.

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 180

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(12, 5))

magdata.plot_lines(data_prop='M_per_mass', data_version='raw', ax=ax[0], legend=True)

magdata.plot_map(
    data_prop='M_per_mass', data_version='raw', ax=ax[1],
    colorbar_kwargs={'ax': ax, 'fraction': 0.05, 'pad': 0.01}
);

## A note about regularization parameters

In general, regularization parameters are dependent on the scale of the data. One might think it
necessary, then, to re-process the data after changing the sample mass or raw data units in order
to appropriately choose regularization parameters. Internally, however, smoothing is done using
the raw values themselves, and conversions happen afterwards. This removes the need to re-process
after conversions.